<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hr_whisper_base_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-18woakhe
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-18woakhe
  Resolved https://github.com/huggingface/transformers to commit 510270af34994c24f2f8792d2afd74114c70d736
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8076097 sha256=7743ea3738c9b422cd6bf9c9e60205183306adf19370e50b87fb9184a6720994
  Stored in directory: /tmp/pip-ephem-wheel-cache-3w8vgp0_/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/

In [6]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [7]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [8]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hr", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 666
})

In [10]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [11]:
model = whisper.load_model("base")

In [12]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [13]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [14]:
text

,0
0,na primjer omogućila je mnogim talentiranim lj...
1,potovani predsjedavajui ribarstvo je vana gosp...
2,nastavlja na temelju ovog europskog projekta k...
3,gospoo predsjedavajua dananje izvjee o europsk...
4,bitno je doprinijeti većem povjerenju potrošač...
...,...
661,radi se o čestim blokadama vlasti napuštanju s...
662,napad islamistikih militanata u sjevernoj nige...
663,ne smijemo ostati slijepi na činjenicu da i eu...
664,u mnogim zemljama ako niste uhvaćeni u počinje...


In [15]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [16]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,female,na primjer omogućila je mnogim talentiranim lj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00286865234375, 0.003021240234375, 0.002655...",16000
1,male,potovani predsjedavajui ribarstvo je vana gosp...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.000152587890625, -0.0006408...",16000
2,male,nastavlja na temelju ovog europskog projekta k...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -0.00018310546875, 9.1552734...",16000
3,male,gospoo predsjedavajua dananje izvjee o europsk...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00360107421875, 0.002685546875, -0.00036621...",16000
4,female,bitno je doprinijeti većem povjerenju potrošač...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.001190185546875, 0.00189...",16000
...,...,...,...,...,...
661,male,radi se o čestim blokadama vlasti napuštanju s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0023193359375, -0.0010986328125, -0.0001831...",16000
662,male,napad islamistikih militanata u sjevernoj nige...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.039825439453125, 0.027557373046875, 0.02493...",16000
663,female,ne smijemo ostati slijepi na činjenicu da i eu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007049560546875, 0.001708984375, -0.0053100...",16000
664,male,u mnogim zemljama ako niste uhvaćeni u počinje...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.000274658203125, 0.00036...",16000


In [17]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
1,male,potovani predsjedavajui ribarstvo je vana gosp...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.000152587890625, -0.0006408...",16000
2,male,nastavlja na temelju ovog europskog projekta k...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -0.00018310546875, 9.1552734...",16000
3,male,gospoo predsjedavajua dananje izvjee o europsk...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00360107421875, 0.002685546875, -0.00036621...",16000
5,male,takva je pozicija neizbjena ne samo zbog injen...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.007781982421875, -0.009033203125, -0.00741...",16000
6,male,uloga europe mora i dalje biti promicanje ljud...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.004669189453125, -0.00543212890625, -0.003...",16000
...,...,...,...,...,...
658,male,stoga sam uvjeren da upravo visoki standardi z...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.009552001953125, -0.018829345703125, -0.01...",16000
659,male,od trenutka pristupanja hrvatske u punopravno ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00042724609375, 0.005462646484375, 0.00509...",16000
661,male,radi se o čestim blokadama vlasti napuštanju s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0023193359375, -0.0010986328125, -0.0001831...",16000
662,male,napad islamistikih militanata u sjevernoj nige...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.039825439453125, 0.027557373046875, 0.02493...",16000


In [18]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
0,female,na primjer omogućila je mnogim talentiranim lj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00286865234375, 0.003021240234375, 0.002655...",16000
4,female,bitno je doprinijeti većem povjerenju potrošač...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.001190185546875, 0.00189...",16000
8,female,dakle sankcije treba uzeti kao mjere koje nisu...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0001220703125, -0.000762939453125, -0.0001...",16000
9,female,ako se ovako nastavi stradat će proizvodnja hr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000396728515625, 0.000152587890625, 0.0006...",16000
11,female,za ostvarivanje ambicioznih ciljeva potrebno j...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001434326171875, 0.0003662109375, -0.000732...",16000
...,...,...,...,...,...
653,female,kao zastupnica iz hrvatske a svjesna alarmantn...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000213623046875, 0.000640869140625, 0.00030...",16000
657,female,niti jedna drava lanica europske unije nije do...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002655029296875, -0.0084228515625, -0.0046...",16000
660,female,kolegice vi jako dobro znate da neke države čl...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00048828125, -0.0006103515625, -0.00018310...",16000
663,female,ne smijemo ostati slijepi na činjenicu da i eu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007049560546875, 0.001708984375, -0.0053100...",16000


In [20]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hr")
    return result["text"]

In [21]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [22]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 1.0
Processed 2 files, WER: 0.5172413793103449
Processed 3 files, WER: 0.8
Processed 4 files, WER: 0.717391304347826
Processed 5 files, WER: 0.7894736842105263
Processed 6 files, WER: 0.9
Processed 7 files, WER: 0.8
Processed 8 files, WER: 0.7241379310344828
Processed 9 files, WER: 0.5897435897435898
Processed 10 files, WER: 0.6875
Processed 11 files, WER: 1.0
Processed 12 files, WER: 0.71875
Processed 13 files, WER: 0.6333333333333333
Processed 14 files, WER: 0.4666666666666667
Processed 15 files, WER: 0.5714285714285714
Processed 16 files, WER: 0.6129032258064516
Processed 17 files, WER: 0.48148148148148145
Processed 18 files, WER: 0.39285714285714285
Processed 19 files, WER: 0.2857142857142857
Processed 20 files, WER: 0.42857142857142855
Processed 21 files, WER: 1.0
Processed 22 files, WER: 1.0
Processed 23 files, WER: 0.6363636363636364
Processed 24 files, WER: 0.6818181818181818
Processed 25 files, WER: 0.4074074074074074
Processed 26 files, WER: 0.88888888

In [23]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.48214285714285715
Processed 2 files, WER: 0.6666666666666666
Processed 3 files, WER: 0.8484848484848485
Processed 4 files, WER: 0.3333333333333333
Processed 5 files, WER: 0.55
Processed 6 files, WER: 0.5833333333333334
Processed 7 files, WER: 0.6
Processed 8 files, WER: 1.1666666666666667
Processed 9 files, WER: 0.7083333333333334
Processed 10 files, WER: 0.5833333333333334
Processed 11 files, WER: 1.0
Processed 12 files, WER: 0.7352941176470589
Processed 13 files, WER: 0.5555555555555556
Processed 14 files, WER: 0.5185185185185185
Processed 15 files, WER: 0.8
Processed 16 files, WER: 0.5882352941176471
Processed 17 files, WER: 0.696969696969697
Processed 18 files, WER: 0.6086956521739131
Processed 19 files, WER: 0.5454545454545454
Processed 20 files, WER: 0.5517241379310345
Processed 21 files, WER: 0.5238095238095238
Processed 22 files, WER: 0.5882352941176471
Processed 23 files, WER: 0.9705882352941176
Processed 24 files, WER: 0.5909090909090909
Processed 25